### GAN CLASSIFIER INFERENCE CODE
Edit the variable checkpoint path and the input file to test the image

In [1]:
import matplotlib
import sys
import numpy as np
import torch
from hparams import create_hparams
from model import GAN_Classifier
from train import load_model
import os
from PIL import Image
import torchvision.transforms as transforms
from utils import to_gpu

### Calculating SRM Features
If SRM input is to be used

In [2]:
# import cv2
# q = [4.0, 12.0, 2.0]
# filter1 = [[0, 0, 0, 0, 0],
#            [0, -1, 2, -1, 0],
#            [0, 2, -4, 2, 0],
#            [0, -1, 2, -1, 0],
#            [0, 0, 0, 0, 0]]
# filter2 = [[-1, 2, -2, 2, -1],
#            [2, -6, 8, -6, 2],
#            [-2, 8, -12, 8, -2],
#            [2, -6, 8, -6, 2],
#            [-1, 2, -2, 2, -1]]
# filter3 = [[0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 0],
#            [0, 1, -2, 1, 0],
#            [0, 0, 0, 0, 0],
#            [0, 0, 0, 0, 0]]
# filter1 = np.asarray(filter1, dtype=float) / q[0]
# filter2 = np.asarray(filter2, dtype=float) / q[1]
# filter3 = np.asarray(filter3, dtype=float) / q[2]

# path='Input Image path'
# dst= 'path_where image is to be saved'   
# filters = filter1+filter2+filter3
# image = cv2.imread(path)
# processed_image = cv2.filter2D(image,-1,filters)
# name='srm'+filename 
# cv2.imwrite(name,processed_image)

In [3]:
hparams=create_hparams()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
checkpoint_path='outdir_resnet_srm/checkpoint_4200'
model=load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval()

### Input image transformation

In [5]:
# img='image_path'
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std= [0.229,0.224,0.225])])
# image=Image.open(img)
# # image.show()
# image=transform(image)
# image=image.unsqueeze(0)
# image=to_gpu(image)
# print(image.size())
# y_pred=model.inference(image)
# _,preds=torch.max(y_pred,1)
# print(y_pred,preds)

### Test set Accuracy 

In [8]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std= [0.229,0.224,0.225])])
path='filelists/gan_images_test_list.txt'
image_set = set((line.strip() for line in open(path)))
accuracy=0.0
total=0.0
i=0
for line in image_set:
    img,srm,y=line.split('|')
    y=torch.tensor(int(y))
    y=y.cuda()

#     image=Image.open(srm)  #if srm is input
    
    image=Image.open(img)  #if original image is input
    image=transform(image)
    image=image.unsqueeze(0)
    image=to_gpu(image)
    y_pred=model.inference(image)
    _,preds=torch.max(y_pred,1)
    accuracy+=torch.sum(preds==y)
    total=total+1
accuracy=accuracy/total
print(accuracy*100,'%')
    

tensor(25.1004, device='cuda:0') %


### Classifying the image